### Libraries

In [1]:
import sys
sys.path.append('../')

In [2]:
import requests 
import json
from dotenv import load_dotenv
import os
import pandas as pd

from functools import reduce
import operator
import geopandas as gdp
import shapely


from pymongo import MongoClient

import src.APIsFunctions as af

In [3]:
load_dotenv()

True

### Creando los geocodes

En este caso, voy a partir de tres ubicaciones diferentes:

- España **-->** Madrid
- Paises Bajos **-->** Amsterdam
- Francia **-->** Paris 
- Canadá **-->** Montréal 

In [4]:
countries = ["madrid", "bilbao", "gijon", "toledo"]

In [5]:
def get_coordinates(lista):
    d = {}
    try: 
        for country in lista:
            locations = requests.get(f'https://geocode.xyz/{country}?json=1')
            locations = locations.json()
            d[country] = locations
    except:
        return "Error getting data"
        
    return d     

In [6]:
c = get_coordinates(countries)

In [7]:
for k, v in c.items():
    print(v.keys())

dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])


In [8]:
def extract_coordinates(dictionary):
    dict_empty = {}
    for k, v in c.items():
        coor = [float(v["longt"]),float(v["latt"])]
        dict_empty[k] = coor
    return dict_empty

In [9]:
coordinates = extract_coordinates(c)

In [10]:
coordinates

{'madrid': [-3.6793, 40.42955],
 'bilbao': [-2.9253, 43.2627],
 'gijon': [-5.7258, 43.5317],
 'toledo': [-4.02711, 39.94034]}

### Llamadas a la API

meter un parametro de radio = metros desde la ubicación dada
meter el parametro de limit para que no me pete, tiene un limite a la hora

In [11]:
token1= os.getenv("token1")
token2 = os.getenv("token2")

In [12]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

#### Llamadas a la API

#####  Codes to queries

Dentro de todas las queries hay algunas que tienen que estar muy cerca o otras que da igual que estén un pco más lejos, lo que haré separarlas por grupos

In [66]:
queries = ["Preschool", "Vegan Restaurant", "Train Station", "Park", "Basketball Court"]

##### Setting the parameters

In [67]:
def get_data (latitude, longitude, *args):
    '''
    descripcion de lo que hace la función aquí
    '''  
    d = {}
    
    for i in args: 
        parametros = {"client_id" : token1,
                  "client_secret" : token2,
                  "v": "20180323",
                  "ll": f"{latitude},{longitude}", 
                  "query":i,
                  "limit": 100}  

        resp = requests.get(url= url_query, params=parametros)
        data = json.loads(resp.text)        
        d[i] = data
        
    return d

In [68]:
madrid = get_data(40.42955, -3.6793, *queries)

In [70]:
toledo = get_data(39.94034, -4.02711, *queries)

In [71]:
bilbao = get_data(43.2627, -2.9253,  *queries )

In [72]:
gijon = get_data(39.94034, -4.02711, *queries)

### Limpieza de los resultados


**Para separar los distintos diccionarios que he genarado con la función anterior:**

In [73]:
def clean_data(result):
    for key, values in result.items():
        
        if key == "Preschool":
            school = result.get(key)
            response = school.get('response')
            decoded = response.get('groups')[0]
            final_school = decoded.get('items')

        elif key == "Vegan Restaurant":
            vegan = result.get(key)
            response = vegan.get('response')
            decoded = response.get('groups')[0]
            final_vegan = decoded.get('items')
            
            
        elif key == "Park":
            park = result.get(key)
            response = park.get('response')
            decoded = response.get('groups')[0]
            final_park = decoded.get('items')
            
        elif key == "Basketball Court":
            sports = result.get(key)
            response = sports.get('response')
            decoded = response.get('groups')[0]
            final_sport = decoded.get('items')
            
        else:
            train = result.get(key)
            response = train.get('response')
            decoded = response.get('groups')[0]
            final_train = decoded.get('items')
            
    results = [final_school, final_vegan,  final_park, final_sport, final_train]
            
    return results

In [74]:
madrid = clean_data(madrid)
bilbao = clean_data(bilbao)
gijon = clean_data(gijon)
toledo = clean_data(toledo)

In [47]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]
m_category = ["venue", "categories"]
    
results = []

for dic in madrid:            
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    paralista["category"] = getFromDict(dic,m_category)

TypeError: string indices must be integers

### Creating dataframes

In [75]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [76]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

####  For school

In [77]:
def create_dataframe (*args):
    mapa_nombre = ["venue","name"]
    m_latitud = ["venue","location","lat"]
    m_longitud = ["venue","location","lng"]
    m_category = ["venue", "categories"]
    
    results = []

    for i in args:
        for dic in i:            
            paralista = {}
            paralista["name"] = getFromDict(dic,mapa_nombre)
            paralista["latitud"] = getFromDict(dic,m_latitud)
            paralista["longitud"] = getFromDict(dic,m_longitud)
            paralista["category"] = getFromDict(dic,m_category)
            results.append(paralista)
    df = pd.DataFrame(results)

    return df

In [78]:
madrid = create_dataframe(*madrid)
bilbao = create_dataframe(*bilbao)
gijon = create_dataframe(*gijon)
toledo = create_dataframe(*toledo)

In [46]:
[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]"

0     [{'id': '52e81612bcbc57f1066b7a45', 'name': 'P...
1     [{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...
2     [{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...
3     [{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...
4     [{'id': '4bf58dd8d48988d116941735', 'name': 'B...
                            ...                        
68    [{'id': '4bf58dd8d48988d12a951735', 'name': 'T...
69    [{'id': '4bf58dd8d48988d12a951735', 'name': 'T...
70    [{'id': '4bf58dd8d48988d12a951735', 'name': 'T...
71    [{'id': '4bf58dd8d48988d1df941735', 'name': 'B...
72    [{'id': '4bf58dd8d48988d129951735', 'name': 'T...
Name: category, Length: 73, dtype: object

###  Saving dataframes

In [79]:

madrid.to_csv("madrid.csv", index = False)

bilbao.to_csv("bilbao.csv", index = False)

gijon.to_csv("gijon.csv", index = False)

toledo.to_csv("toledo.csv", index = False)

